In [49]:
from bs4 import BeautifulSoup
import urllib
import urllib.error
import urllib.request
import json
import pandas as pd
import numpy as np


URL_HOME = 'https://www.winamax.fr/paris-sportifs'
URL_MATCH = 'https://www.winamax.fr/paris-sportifs/match/'
URL_SPORT = 'https://www.winamax.fr/paris-sportifs/sports/'
URL_BOOST = 'https://www.winamax.fr/paris-sportifs/sports/100000'


def request_get_html(url):
    req = urllib.request.Request(
        url, 
        headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
    })
    page = urllib.request.urlopen(req, timeout=10).read()
    parser = BeautifulSoup(page, 'html.parser')
    return parser

def winamax_get_dict_data(html):
    for line in html.find_all(['script']):
        if "PRELOADED_STATE" in str(line.string):
            txt_line = line.string
            txt_line = txt_line.split('var PRELOADED_STATE = ')[1][:-1]
            d_line = json.loads(txt_line)
    return d_line


## Retrieve list sports ID
req = request_get_html(URL_HOME)
d_data = winamax_get_dict_data(req)
l_sport_ids = d_data['sportIds']
l_sport_ids.remove(100000)
arr_sport_ids = np.array(l_sport_ids)
d_mapping_sport = {}
for k in d_data['sports']:
    d_mapping_sport[k] = d_data['sports'][str(k)]['sportName']


In [68]:
class Sport:
    def __init__(self, id, name):
        self.id = id
        self.name = name
        
        
class Match:
    def __init__(self, matchId, available, status, title, 
                 sportId, categoryId, tournamentId, 
                 competitor1Id, competitor1Name, 
                 competitor2Id, competitor2Name):
        self.matchId = matchId
        self.available = available
        self.status = status
        self.title = title
        self.sportId = sportId
        self.categoryId = categoryId
        self.tournamentId = tournamentId
        self.competitor1Id = competitor1Id
        self.competitor1Name = competitor1Name
        self.competitor2Id = competitor2Id
        self.competitor2Name = competitor2Name
    

class Bet:
    def __init__(self, betId, marketId, outcomes,
                 betTypeIsLive, betTitle, betTypeName,
                 betType, betTypeCategoryId, betTypeCategory, betGroup):
        self.betId = betId
        self.marketId = marketId
        self.outcomes = outcomes
        self.betTypeIsLive = betTypeIsLive
        self.betTitle = betTitle
        self.betTypeName = betTypeName
        self.betType = betType
        self.betTypeCategoryId = betTypeCategoryId
        self.betTypeCategory = betTypeCategory
        self.betGroup = betGroup

In [75]:
df_matches = pd.DataFrame()
for sport_id in arr_sport_ids:
    url = URL_SPORT + str(sport_id)
    req = request_get_html(url)
    d_data = winamax_get_dict_data(req)
    l_match = []
    for match in d_data['matches']:
        row = d_data['matches'][match]
        Match(row['matchId'], row['available'], row['status'], row['title'],
              row['sportId'], row['categoryId'], row['tournamentId'], row['competitor1Id'],
              row['competitor1Name'], row['competitor2Id'], row['competitor2Name'])
        break
    break

{'matchId': 27298822, 'available': True, 'status': 'PREMATCH', 'mainBetId': 116400254, 'moreBets': 33, 'periodId': 0, 'period': 'not_started', 'periodName': '', 'hlType': 'edito', 'title': 'Novak Djokovic - Alejandro Davidovich-Fokina', 'roundName': '1/8 de finale', 'tvChannels': 'WINAMAXTV', 'sportId': 5, 'categoryId': 3, 'tournamentId': 98716, 'competitor1Id': 14882, 'competitor1Name': 'Novak Djokovic', 'competitor2Id': 157456, 'competitor2Name': 'Alejandro Davidovich-Fokina', 'matchStart': 1620892800, 'liveodds': 'bookable', 'prematchDisplayOrder': None, 'liveDisplayOrder': None, 'filters': [106, 107, 108, 110, 112, 113, 115, 116, 117, 118, 119, 120, 122, 123, 131, 132]}


In [70]:
d_data

{'sportIds': [100000,
  1,
  5,
  2,
  11,
  3,
  10,
  17,
  40,
  16,
  13,
  9,
  6,
  4,
  12,
  9992,
  14,
  23],
 'sports': {'1': {'sportName': 'Football',
   'categories': [7,
    30,
    1,
    32,
    31,
    44,
    800000006,
    800000014,
    800000007,
    800000009,
    800000288,
    800000420,
    800000073,
    800000361,
    800000119,
    34,
    17,
    33,
    91,
    78,
    99,
    291,
    14,
    8,
    22,
    165,
    92,
    26,
    19,
    67,
    201,
    51,
    130,
    10,
    66,
    52,
    278,
    163,
    303,
    12,
    5,
    35,
    131,
    47,
    18,
    77,
    21,
    23,
    24,
    9,
    46],
   'mainMatchCount': 407,
   'liveMatchCount': 8,
   'tvMatchCount': 254,
   'filters': [1,
    2,
    3,
    4,
    5,
    6,
    7,
    8,
    11,
    12,
    13,
    14,
    15,
    16,
    17,
    18,
    19,
    20,
    21,
    22,
    23,
    24,
    25,
    26,
    27,
    28,
    29,
    30,
    31,
    32,
    33,
    34,
    35,
    36,

In [72]:
d_data['matches']

{'27298822': {'matchId': 27298822,
  'available': True,
  'status': 'PREMATCH',
  'mainBetId': 116400254,
  'moreBets': 33,
  'periodId': 0,
  'period': 'not_started',
  'periodName': '',
  'hlType': 'edito',
  'title': 'Novak Djokovic - Alejandro Davidovich-Fokina',
  'roundName': '1/8 de finale',
  'tvChannels': 'WINAMAXTV',
  'sportId': 5,
  'categoryId': 3,
  'tournamentId': 98716,
  'competitor1Id': 14882,
  'competitor1Name': 'Novak Djokovic',
  'competitor2Id': 157456,
  'competitor2Name': 'Alejandro Davidovich-Fokina',
  'matchStart': 1620892800,
  'liveodds': 'bookable',
  'prematchDisplayOrder': None,
  'liveDisplayOrder': None,
  'filters': [106,
   107,
   108,
   110,
   112,
   113,
   115,
   116,
   117,
   118,
   119,
   120,
   122,
   123,
   131,
   132]},
 '27299430': {'matchId': 27299430,
  'available': True,
  'status': 'PREMATCH',
  'mainBetId': 116428657,
  'moreBets': 33,
  'periodId': 0,
  'period': 'not_started',
  'periodName': '',
  'hlType': 'to_come',
 

In [71]:
d_data.keys()

dict_keys(['sportIds', 'sports', 'categories', 'tournaments', 'calendar', 'filters', 'teasers', 'matches', 'bets', 'outcomes', 'odds', 'settings', 'cc', 'browser', 'notifications', 'terms', 'events'])

In [67]:
d_data['bets']

{'115859774': {'betId': 115859774,
  'marketId': 186,
  'outcomes': [377096949, 377096950],
  'available': True,
  'template': '2way',
  'isAlternativeMainBet': False,
  'betTypeIsLive': False,
  'betTitle': 'Vainqueur',
  'betTypeName': 'Vainqueur',
  'betType': 3624,
  'betTypeCategoryId': 2,
  'betTypeCategory': 'Match',
  'betTypeHelp': 'Pariez sur le vainqueur du match. ID:3624',
  'betGroup': 1},
 '115859945': {'betId': 115859945,
  'marketId': 186,
  'outcomes': [377097356, 377097357],
  'available': True,
  'template': '2way',
  'isAlternativeMainBet': False,
  'betTypeIsLive': False,
  'betTitle': 'Vainqueur',
  'betTypeName': 'Vainqueur',
  'betType': 3624,
  'betTypeCategoryId': 2,
  'betTypeCategory': 'Match',
  'betTypeHelp': 'Pariez sur le vainqueur du match. ID:3624',
  'betGroup': 1},
 '115859971': {'betId': 115859971,
  'marketId': 186,
  'outcomes': [377097417, 377097418],
  'available': True,
  'template': '2way',
  'isAlternativeMainBet': False,
  'betTypeIsLive': F

In [59]:
req = request_get_html('https://www.winamax.fr/paris-sportifs/sports/5')
d_data = winamax_get_dict_data(req)

In [63]:
d_data['matches']

{'27245534': {'matchId': 27245534,
  'available': True,
  'status': 'PREMATCH',
  'mainBetId': 115859971,
  'moreBets': 33,
  'periodId': 0,
  'period': 'not_started',
  'periodName': '',
  'hlType': 'to_come',
  'title': 'Ugo Humbert - Jannik Sinner',
  'roundName': None,
  'tvChannels': None,
  'sportId': 5,
  'categoryId': 3,
  'tournamentId': 98716,
  'competitor1Id': 185388,
  'competitor1Name': 'Ugo Humbert',
  'competitor2Id': 225050,
  'competitor2Name': 'Jannik Sinner',
  'matchStart': 1620642900,
  'liveodds': 'bookable',
  'prematchDisplayOrder': None,
  'liveDisplayOrder': None,
  'filters': [106,
   107,
   108,
   110,
   112,
   113,
   115,
   116,
   117,
   118,
   119,
   120,
   122,
   123,
   131,
   132]},
 '27245536': {'matchId': 27245536,
  'available': True,
  'status': 'PREMATCH',
  'mainBetId': 115859945,
  'moreBets': 33,
  'periodId': 0,
  'period': 'not_started',
  'periodName': '',
  'hlType': 'to_come',
  'title': 'Stefano Travaglia - Benoit Paire',
  '

In [65]:
d_data['bets']

{'115859774': {'betId': 115859774,
  'marketId': 186,
  'outcomes': [377096949, 377096950],
  'available': True,
  'template': '2way',
  'isAlternativeMainBet': False,
  'betTypeIsLive': False,
  'betTitle': 'Vainqueur',
  'betTypeName': 'Vainqueur',
  'betType': 3624,
  'betTypeCategoryId': 2,
  'betTypeCategory': 'Match',
  'betTypeHelp': 'Pariez sur le vainqueur du match. ID:3624',
  'betGroup': 1},
 '115859945': {'betId': 115859945,
  'marketId': 186,
  'outcomes': [377097356, 377097357],
  'available': True,
  'template': '2way',
  'isAlternativeMainBet': False,
  'betTypeIsLive': False,
  'betTitle': 'Vainqueur',
  'betTypeName': 'Vainqueur',
  'betType': 3624,
  'betTypeCategoryId': 2,
  'betTypeCategory': 'Match',
  'betTypeHelp': 'Pariez sur le vainqueur du match. ID:3624',
  'betGroup': 1},
 '115859971': {'betId': 115859971,
  'marketId': 186,
  'outcomes': [377097417, 377097418],
  'available': True,
  'template': '2way',
  'isAlternativeMainBet': False,
  'betTypeIsLive': F

In [64]:
d_data.keys()

dict_keys(['sportIds', 'sports', 'categories', 'tournaments', 'calendar', 'filters', 'teasers', 'matches', 'bets', 'outcomes', 'odds', 'settings', 'cc', 'browser', 'notifications', 'terms', 'events'])

In [30]:
for line in parser.find_all(['script']):
    if "PRELOADED_STATE" in str(line.string):
        txt_line = line.string
        txt_line = txt_line.split('var PRELOADED_STATE = ')[1][:-1]
        d_line = json.loads(txt_line)

In [31]:
d_line.keys()

dict_keys(['sportIds', 'sports', 'categories', 'tournaments', 'calendar', 'filters', 'teasers', 'matches', 'bets', 'outcomes', 'odds', 'settings', 'home', 'cc', 'browser', 'notifications', 'terms', 'events'])

In [33]:
d_line['sportIds'].remove(100000)

In [34]:
d_line

{'sportIds': [1, 5, 2, 11, 3, 10, 17, 40, 16, 13, 9, 6, 4, 12, 9992, 14, 23],
 'sports': {'1': {'sportName': 'Football',
   'categories': [800000007,
    7,
    30,
    1,
    32,
    31,
    44,
    800000006,
    800000014,
    800000009,
    800000288,
    800000420,
    800000073,
    800000361,
    257,
    34,
    17,
    33,
    91,
    158,
    78,
    49,
    99,
    102,
    274,
    291,
    14,
    8,
    22,
    165,
    92,
    26,
    19,
    938,
    67,
    11,
    51,
    10,
    66,
    52,
    160,
    303,
    12,
    5,
    280,
    35,
    131,
    47,
    18,
    77,
    21,
    23,
    9,
    25,
    46],
   'mainMatchCount': 493,
   'liveMatchCount': 5,
   'tvMatchCount': 244,
   'filters': [1,
    2,
    3,
    4,
    5,
    6,
    7,
    8,
    11,
    12,
    13,
    14,
    15,
    16,
    17,
    18,
    19,
    20,
    21,
    22,
    23,
    24,
    25,
    26,
    27,
    28,
    29,
    30,
    31,
    32,
    34,
    35,
    36,
    37,
    38,
    4

In [27]:
len(d_line['matches'].keys())

46

In [76]:
d_line['matches']['27182860']

KeyError: '27182860'

In [ ]:
d_line['bets'] 186

In [62]:
d_line['bets']

{'114328679': {'betId': 114328679,
  'marketId': 1,
  'outcomes': [371421411, 371421412, 371421413],
  'available': True,
  'template': '3way',
  'isAlternativeMainBet': False,
  'betTypeIsLive': False,
  'betTitle': 'Résultat',
  'betTypeName': 'Résultat',
  'betType': 3178,
  'betTypeCategoryId': 1,
  'betTypeCategory': 'Résultat',
  'betTypeHelp': 'Pariez sur le résultat du match (hors prolongations et tirs au but éventuels). ID:3178',
  'betGroup': 1},
 '115144564': {'betId': 115144564,
  'marketId': 186,
  'outcomes': [374403657, 374403658],
  'available': True,
  'template': '2way',
  'isAlternativeMainBet': False,
  'betTypeIsLive': False,
  'betTitle': 'Vainqueur',
  'betTypeName': 'Vainqueur',
  'betType': 3624,
  'betTypeCategoryId': 2,
  'betTypeCategory': 'Match',
  'betTypeHelp': 'Pariez sur le vainqueur du match. ID:3624',
  'betGroup': 1},
 '115188710': {'betId': 115188710,
  'marketId': 186,
  'outcomes': [374596284, 374596285],
  'available': True,
  'template': '2way'

In [19]:
d_line['outcomes']['370532640']

{'label': 'R. Sociedad',
 'available': True,
 'code': '1',
 'percentDistribution': 95}

In [63]:
d_line['odds']['374403683']

1.48

In [59]:
d_line['matches']

{'26890590': {'matchId': 26890590,
  'available': True,
  'status': 'PREMATCH',
  'mainBetId': 114328679,
  'moreBets': 108,
  'periodId': 0,
  'period': 'not_started',
  'periodName': '',
  'hlType': 'edito',
  'title': 'Chelsea - Real Madrid',
  'roundName': 'Demi-finale - Match retour',
  'tvChannels': 'RMCSPORT1',
  'sportId': 1,
  'categoryId': 800000006,
  'tournamentId': 23,
  'competitor1Id': 38,
  'competitor1Name': 'Chelsea',
  'competitor2Id': 2829,
  'competitor2Name': 'Real Madrid',
  'matchStart': 1620241200,
  'liveodds': 'booked',
  'prematchDisplayOrder': None,
  'liveDisplayOrder': None,
  'filters': [1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38,
   40,
   41,
   42,
   43,
   45,
   46,
   47,
   48,
   49,
   50,
   52,
   53,
   55,
   56,
   57,
   58,
   59,
   60,
   61,
   62,